# Edward as visualized by Moana on May 8

### Important note
It takes <font color='red'>over 50 minutes</font> to run this notebook! Especially line [7]. There is no way around it, I confirmed it with NASA.

### Contents
1. Setup
2. Edward zoomed-out
3. Create dataset with lat long coordinates
4. Edward zoomed-in (NOT based on lat long coordinates)

# 1. Setup

In [22]:
# Import libraries and functions
import csv
import os

import cartopy.crs as ccrs
import earthaccess
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import subprocess

#### Note on function to help write OCSSW parameter files
To write the results in the format understood by OCSSW, this function uses the `csv.writer` from the Python Standard Library. Instead of writing comma-separated values, however, we specify a non-default delimiter to get equals-separated values. Not something you usually see in a data file, but it's better than
writing our own utility from scratch! Learn more under: #help(write_par)

In [5]:
earthaccess.login()

In [6]:
# Define function to help write parameter files
def write_par(path, par):
    """
    Prepare a "par file" to be read by one of the OCSSW tools, as an
    alternative to specifying each parameter on the command line.

    Args:
        path (str): where to write the parameter file
        par (dict): the parameter names and values included in the file
    """
    with open(path, "w") as file:
        writer = csv.writer(file, delimiter="=")
        values = writer.writerows(par.items())

Get list of dates to run MOANA 

In [8]:
df = pd.read_csv("CHL_TIMESERIES_PACE.csv")
dates=df['satdate'].values

# 2. Run through dates above, MOANA images of interest

#### Note on l2bin:
It is very slow, takes <font color='red'>over 50 minutes</font> to run.
There is <font color='red'>no way to pre-define a boundary box upfront</font>: the entire region has to run and then we have to crop.

In [29]:
os.getcwd()

'/home/jovyan/GO-SWACE'

In [25]:
for i in range(0,len(dates)):

    #First, define date and location to search for Edward
    sel=str(dates[i])
    date = datetime.strptime(sel, '%Y%m%d').strftime('%m/%d/%Y')
    begindate=sel[0:4]+'-'+sel[4:6]+'-'+sel[6:8]
    
    input=date
    format = '%m/%d/%Y'
    datetime = datetime.strptime(input, format)
    datetime+= timedelta(days=1)
    enddate=str(datetime.date())

    tspan = (begindate, enddate)
    location = (-70, 36) # We can not use Boundary box, as l2gen can not process that information
    
    results = earthaccess.search_data(
        short_name="PACE_OCI_L1B_SCI",
        temporal=tspan,
        point=location,
    )
    results[0]

    #Open and download PACE data
    paths = earthaccess.download(results, local_path="data")
    #open dataset
    dataset = xr.open_dataset(paths[0], group="observation_data")

    # Define parameters
    ifile = paths[0]
    par = {
        "ifile": ifile,
        "ofile": str(ifile).replace(".L1B.", ".L2_MOANA."),
        "suite": "BGC",
        "l2prod": "picoeuk_moana prococcus_moana syncoccus_moana rhos_465 rhos_555 rhos_645 poc chlor_a ",
        "atmocor": 1,
    }
    write_par("l2gen-moana.par", par)

    #Execute short bash script to run MOANA
    subprocess.run(["bash", "executemoana.sh"])
    
    #Extract data and save as netcdf
    dataset_geophysical = xr.open_dataset(par["ofile"], group="geophysical_data")

    # Create dataset with lat long coordinates
    dataset_navigation = xr.open_dataset(par["ofile"], group="navigation_data")
    dataset_merged = xr.merge((dataset, nav))

    # Download file as netCDF
    datasetname='moana'+sel
    dataset.to_netcdf(datasetname)

Granules found: 1
 Getting 1 granules, approx download size: 1.73 GB
Accessing cloud dataset using dataset endpoint credentials: https://obdaac-tea.earthdatacloud.nasa.gov/s3credentials
Downloaded: data/PACE_OCI.20240426T171515.L1B.V2.nc


FileNotFoundError: [Errno 2] No such file or directory: 'executemoana.sh'